In [3]:
import pickle
with open("../../../Data/Processed/XY_formats/One_hot_encoded_up_down_multi_pytorch.pkl", "rb") as f:
    xy_dict = pickle.load(f)

In [22]:
print(len(xy_dict))

143745


In [9]:
print(xy_dict[0][0][0].shape)

(1, 4001, 4)


In [11]:
import numpy as np
gene_names = np.load("../../../Data/Processed/XY_formats/gene_names_one_hot.npy")

In [20]:
print(len(gene_names))

143745


In [12]:
def family_wise_train_test_splitting(
    gene_families: np.array, test_size=0.2, random_state=42
):
    """
    Split the data into training and testing sets, ensuring that the gene families are not split between the two sets.

    gene_families: The gene families
    test_size: The proportion of the data to be used as the test set
    random_state: The random state for the shuffle split
    ---

    returns: Training and test indices

    """

    X = np.zeros(shape=(len(gene_families), 1))
    Y = np.zeros(shape=(len(gene_families), 1))

    # Create the split
    gss = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)

    # Split the data
    train_index, test_index = next(gss.split(X, Y, groups=gene_families))

    # assert no family is present in both sets
    assert (
        len(
            set(gene_families[train_index]).intersection(set(gene_families[test_index]))
        )
        == 0
    )

    # Return the split data indices
    return train_index, test_index

In [15]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit, GroupKFold
X_sequence = np.array(
    [xy_dict[gene][0][0].T for gene in xy_dict]
).squeeze()  # [random_index]
X_treatment = np.array([xy_dict[gene][0][1] for gene in xy_dict])  # [random_index]
Y = np.array([xy_dict[gene][1] for gene in xy_dict])  # [random_index]

# Get family names per gene
# load gene families
gene_families = pd.read_csv("../../../Data/Processed/gene_families.csv", index_col=1)

# Map gene names to family names
# I cannot use a dictionary because the genes are repeated 5 times, eachper treatment
# I will use a list instead
family_names = []
for gene in gene_names:
    family_names.append(gene_families.loc[gene, "family_id"])


# Split the data
#test 1
#test1 2
# train_index, test_index = family_wise_train_test_splitting(X_treatment, Y, np.array(family_names)[random_index], random_state = 7215, return_index=True)
train_index, test_index = family_wise_train_test_splitting(
     np.array(family_names), random_state=3
)

In [16]:
train_index, val_index = family_wise_train_test_splitting(
    np.array(family_names)[train_index],
    random_state=4,
    test_size=0.3,
)

In [17]:
X_train_sequence = X_sequence[train_index]
X_test_sequence = X_sequence[test_index]
X_train_treatment = X_treatment[train_index]
X_test_treatment = X_treatment[test_index]
Y_train = Y[train_index]
Y_test = Y[test_index]
X_val_sequence = X_sequence[val_index]
X_val_treatment = X_treatment[val_index]
Y_val = Y[val_index]

In [18]:
print(X_train_sequence.shape)

(83470, 4, 4001)
